In [9]:
import pandas as pd
import numpy as np
import requests
import time

In [3]:
#base URLS for using pushshift to query submissions and comments.

sub_url= "https://api.pushshift.io/reddit/search/submission/"
com_url= "https://api.pushshift.io/reddit/search/comment/"

 Let's look at submitted comments with a score of at least 50. I want to focus on comments rather than posts, as the bulk of posts are just links -- I think there will be more opinionated text to analyze in comments.



PushShift only lets you grab 100 records at a time. Let's get timestamps of weeks for the two years, and then make calls for each of them. The timestamp for Sept 30, 2020, 11:59 PM is 1601510399

In [4]:
#let's calculate the seconds in a week.
secs=60*60*24*7
secs

604800

In [5]:
weekly_timestamps=[]
x= 1601510399
for i in range(105):
    weekly_timestamps.append(x)
    x-=secs

In [6]:
#this is a list of 104 timestamp pairs -- the week end, and the week beginning
timestamp_pairs= [(weekly_timestamps[i], weekly_timestamps[i+1]) for i in range(104)] 

In [7]:
timestamp_pairs

[(1601510399, 1600905599),
 (1600905599, 1600300799),
 (1600300799, 1599695999),
 (1599695999, 1599091199),
 (1599091199, 1598486399),
 (1598486399, 1597881599),
 (1597881599, 1597276799),
 (1597276799, 1596671999),
 (1596671999, 1596067199),
 (1596067199, 1595462399),
 (1595462399, 1594857599),
 (1594857599, 1594252799),
 (1594252799, 1593647999),
 (1593647999, 1593043199),
 (1593043199, 1592438399),
 (1592438399, 1591833599),
 (1591833599, 1591228799),
 (1591228799, 1590623999),
 (1590623999, 1590019199),
 (1590019199, 1589414399),
 (1589414399, 1588809599),
 (1588809599, 1588204799),
 (1588204799, 1587599999),
 (1587599999, 1586995199),
 (1586995199, 1586390399),
 (1586390399, 1585785599),
 (1585785599, 1585180799),
 (1585180799, 1584575999),
 (1584575999, 1583971199),
 (1583971199, 1583366399),
 (1583366399, 1582761599),
 (1582761599, 1582156799),
 (1582156799, 1581551999),
 (1581551999, 1580947199),
 (1580947199, 1580342399),
 (1580342399, 1579737599),
 (1579737599, 1579132799),
 

In [8]:


all_weeks_ogft=[]
for pair in timestamp_pairs:
    before_ts=pair[0]
    after_ts=pair[1]
    ogft_params={'subreddit':'onguardforthee',
           'after': after_ts,
           'before':before_ts,
            'sort_type': 'score',
            'sort':'desc',
            'size':100}
    req_ogft=requests.get(com_url, params=ogft_params)
    print(pair)
    while req_ogft.status_code != 200:
        time.sleep(5)
        req_ogft=requests.get(com_url, params=ogft_params)
    all_weeks_ogft.append(req_ogft.json())
    time.sleep(2)

(1601510399, 1600905599)
(1600905599, 1600300799)
(1600300799, 1599695999)
(1599695999, 1599091199)
(1599091199, 1598486399)
(1598486399, 1597881599)
(1597881599, 1597276799)
(1597276799, 1596671999)


KeyboardInterrupt: 

In [ ]:
x=0
for week in all_weeks_ogft:
    if len(week['data']) != 100:
        print(f"week {x} incomplete")
        x+=1

In [ ]:

all_weeks_can=[]
for pair in timestamp_pairs:
    before_ts=pair[0]
    after_ts=pair[1]
    can_params={'subreddit':'canada',
           'after': after_ts,
           'before':before_ts,
            'sort_type': 'score',
            'sort':'desc',
            'size':100}
    req_can=requests.get(com_url, params=can_params)
    print(pair)
    while req_can.status_code != 200:
        time.sleep(5)
        req_can=requests.get(sub_url, params=can_params)
    all_weeks_can.append(req_can.json())
    time.sleep(2)

In [ ]:
x=0
for week in all_weeks_can:
    if len(week['data']) != 100:
        print(f"week {x} incomplete")
        x+=1

Disclaimer: I was hoping to do some score based filtering on the comments. However, putting anything in the score filter for the API produced unexpected results.

In [ ]:
all_weeks_ogft.extend(all_weeks_can)

In [ ]:
all_weeks=all_weeks_ogft

In [ ]:
len(all_weeks)

In [ ]:
all_weeks_dfs=[pd.DataFrame(week['data']) for week in all_weeks]

In [ ]:
all_weeks_dfs[0]['author'].value_counts()

In [ ]:
reddit_df=pd.concat(all_weeks_dfs)

In [ ]:
reddit_df.columns

We can toss away most of this information. Things that we want. Mostly we just need 'subreddit' and 'body'. But let's also pull in score, controversiality, and created_utc for some explanatory stats.

In [ ]:
canada_df=reddit_df[['subreddit','author','created_utc','score','controversiality', 'body']].copy()

In [ ]:
# fix the busted indices
canada_df.reset_index(drop=True, inplace=True)

In [ ]:
#on second thought, controversiality isn't doing anthing for us.
canada_df.drop(columns=['controversiality'], inplace=True)

In [ ]:
canada_df['body'].value_counts()

Let's get rid of deleted or removed messages.

In [ ]:
mask = (canada_df['body']==f'[removed]') | (canada_df['body']==f'[deleted]')

In [ ]:
canada_df.drop(canada_df[mask].index, inplace=True)

In [ ]:
canada_df[canada_df['body'].str.contains('Thank you for your submission')]

In [ ]:
mask= canada_df['body'].str.contains('Thank you for your submission')
canada_df.drop(canada_df[mask].index, inplace=True)

In [13]:
mask= canada_df['author']=='AutoModerator'
canada_df.drop(canada_df[mask].index, inplace=True)

In [14]:
canada_df.reset_index(drop=True, inplace=True)

In [15]:
canada_df.shape

(20102, 7)

In [16]:
canada_df['subreddit'].value_counts(normalize=True)

onguardforthee    0.50388
canada            0.49612
Name: subreddit, dtype: float64

I think we're about ready to process this. Let's export the data and proceed to the next notebook.

In [17]:
canada_df.to_csv('..\data\canada_subreddit_comments.csv', index=False)